In [1]:
!pip install flask pyngrok yt-dlp pydub
!apt-get install ffmpeg -y


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [ ]:

from flask import Flask, request, render_template_string
from pyngrok import ngrok
import os, zipfile, smtplib
from email.message import EmailMessage
from yt_dlp import YoutubeDL
from pydub import AudioSegment

app = Flask(__name__)

HTML = """
<!DOCTYPE html>
<html>
<head>
<title>Mashup Generator</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: #f5dede;
    margin: 0;
    padding: 0;
}

.container {
    width: 420px;
    margin: 60px auto;
    background: white;
    padding: 35px;
    border-radius: 15px;
    box-shadow: 0px 4px 18px rgba(0,0,0,0.2);
}

h2 {
    text-align: center;
    margin-bottom: 25px;
}

label {
    font-weight: bold;
    display: block;
    margin-top: 15px;
}

input {
    width: 100%;
    padding: 12px;
    margin-top: 6px;
    border-radius: 8px;
    border: 1px solid #ccc;
    font-size: 14px;
}

button {
    width: 100%;
    margin-top: 25px;
    padding: 14px;
    border: none;
    border-radius: 8px;
    background: linear-gradient(90deg,#2d6cdf,#3d8bfd);
    color: white;
    font-size: 16px;
    cursor: pointer;
}

button:hover {
    background: linear-gradient(90deg,#1f5bd0,#2d6cdf);
}
</style>
</head>

<body>

<div class="container">

<h2>Mashup Generator</h2>

<form method="post">

<label>Singer Name</label>
<input name="singer" required>

<label>Number of Videos</label>
<input name="num" required>

<label>Duration (seconds)</label>
<input name="duration" required>

<label>Email ID</label>
<input name="email" required>

<button type="submit">Generate Mashup</button>

</form>

</div>

</body>
</html>
"""


def download_audio(singer, num):
    os.makedirs("downloads", exist_ok=True)
    query = f"ytsearch{num}:{singer}"
    opts = {'format':'bestaudio','outtmpl':'downloads/%(title)s.%(ext)s'}
    with YoutubeDL(opts) as ydl:
        ydl.download([query])

def create_mashup(duration):
    final = AudioSegment.empty()
    for f in os.listdir("downloads"):
        audio = AudioSegment.from_file("downloads/"+f)
        final += audio[:duration*1000]
    final.export("output.mp3", format="mp3")

def send_email(receiver):
    sender = "siyakhosla93@gmail.com"
    password = "faeseurpikvpuast"

    with zipfile.ZipFile("mashup.zip","w") as z:
        z.write("output.mp3")

    msg = EmailMessage()
    msg["Subject"] = "Your Mashup File"
    msg["From"] = sender
    msg["To"] = receiver
    msg.set_content("Your mashup is attached.")

    with open("mashup.zip","rb") as f:
        msg.add_attachment(f.read(), maintype="application", subtype="zip", filename="mashup.zip")

    with smtplib.SMTP_SSL("smtp.gmail.com",465) as smtp:
        smtp.login(sender,password)
        smtp.send_message(msg)

@app.route("/",methods=["GET","POST"])
def home():
    if request.method=="POST":
        singer = request.form["singer"]
        num = int(request.form["num"])
        duration = int(request.form["duration"])
        email = request.form["email"]

        download_audio(singer,num)
        create_mashup(duration)
        send_email(email)

        return "Mashup created & emailed successfully!"
    return render_template_string(HTML)

url = ngrok.connect(5000)
print("OPEN THIS LINK:",url)

app.run(port=5000)


OPEN THIS LINK: NgrokTunnel: "https://pectinate-drivellingly-phuong.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Feb/2026 16:40:56] "GET / HTTP/1.1" 200 -


[youtube:search] Extracting URL: ytsearch2:Arijit Singh
[download] Downloading playlist: Arijit Singh
[youtube:search] query "Arijit Singh": Downloading web client config
[youtube:search] query "Arijit Singh" page 1: Downloading API JSON
[youtube:search] Playlist Arijit Singh: Downloading 2 items of 2
[download] Downloading item 1 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=O5gwxm3NxFU
[youtube] O5gwxm3NxFU: Downloading webpage


[youtube] O5gwxm3NxFU: Downloading android vr player API JSON
[info] O5gwxm3NxFU: Downloading 1 format(s): 251
[download] downloads/Best Of Arijit Singh 2024 ｜ Arijit Singh Hits Songs ｜ Arijit Singh Jukebox Songs ｜ Indian Songs.webm has already been downloaded
[download] 100% of   55.13MiB
[download] Downloading item 2 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=llLmk_eYHwE
[youtube] llLmk_eYHwE: Downloading webpage


[youtube] llLmk_eYHwE: Downloading android vr player API JSON
[info] llLmk_eYHwE: Downloading 1 format(s): 251
[download] downloads/Best of Arijit Singh Songs ｜ 40 Super Hit Songs ｜ 2+ Hours Non-Stop ｜ Romantic Hits.webm has already been downloaded
[download] 100% of  122.81MiB
[download] Finished downloading playlist: Arijit Singh


INFO:werkzeug:127.0.0.1 - - [12/Feb/2026 16:43:09] "POST / HTTP/1.1" 200 -


In [4]:
from pyngrok import ngrok

ngrok.set_auth_token("39ArZ1ei9gFBkLd5cF1K5GZBNyb_6WZigHtD6R3SHVLQyYQfe")
